In [1]:
import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from pprint import PrettyPrinter as ppr
from cimaq_utils import emptydir
from cimaq_utils import flatten
from cimaq_utils import loadimages

from unzipper import unzipper
from NEW_extract_events2 import NEW_extract_events2
from NEW_get_allsheetsdf import NEW_get_allsheetsdf
from NEW_sniffer2 import NEW_sniffer2
from sniffzipped import sniffzipped

# set max display size options for pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
# from scent import scent

# from NEW_sniff_encodtype import NEW_sniff_encodtype
from chardet.universaldetector import UniversalDetector as udet


In [12]:
# There's no need to change current working directory
# It's all relative
maindir = xpu("~/extracted_eprime0")

# Load DataFrame outputted during extraction
p1 = pd.read_csv(join(maindir,"participants.tsv"),
                 sep='\t').set_index("sub-ID")

display(p1.head())

,pscid,dccid,zfilename,zipdirpaths,extdirpaths,nitems
sub-ID,,,,,,
sub-3025432,3025432,658178,3025432_658178_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime/3025432_658178_Tache_IRM.zip,/home/fnadeau/extracted_eprime0/sub-3025432/archive,1
sub-3123186,3123186,920577,3123186_920577_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime/3123186_920577_Tache_IRM.zip,/home/fnadeau/extracted_eprime0/sub-3123186/archive,1
sub-3163875,3163875,199801,3163875_199801_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime/3163875_199801_Tache_IRM.zip,/home/fnadeau/extracted_eprime0/sub-3163875/archive,1
sub-3291977,3291977,748676,3291977_748676_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime/3291977_748676_Tache_IRM.zip,/home/fnadeau/extracted_eprime0/sub-3291977/archive,1
sub-3303819,3303819,893639,3303819_893639_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime/3303819_893639_Tache_IRM.zip,/home/fnadeau/extracted_eprime0/sub-3303819/archive,1


In [6]:
# There's no need to change current working directory
# It's all relative
maindir = xpu("~/extracted_eprime0")

# Load DataFrame outputted during extraction
p1 = pd.read_csv(join(maindir,"participants.tsv"),
                 sep='\t').set_index("sub-ID")

# List files extracted for each participant
p1['sheets'] = [[item for item in ls(join(maindir, row[0]))
                if os.path.isfile(join(maindir, row[0], item))]
                for row in p1.iterrows()]

# List each of previous step's paths
p1['sheetpaths'] = [[join(maindir, row[0], item)
                      for item in ls(join(maindir, row[0]))
                     if os.path.isfile(join(maindir, row[0], item))]
                     for row in p1.iterrows()]
test = [pd.read_fwf(sheetpath) for sheetpath in p1.iloc[44]["sheetpaths"]]

display(test)


# get unique file names
# ufnames = sorted(list(dict.fromkeys([os.path.splitext(bname(file))[0].split(
#     c for c in chr(os.path.splitext(bname(file))[0]) if c.isdigit())
#                                     for file in flatten(p1["sheets"])])))
# # test2 = dict((testkey, json.loads(testval, 'str'))
# #             for testkey, testval in test.items())
# # display(test2)
# # p1["extitempaths"] = [row[1]["extitempaths"][0] for row in p1.iterrows()]
# # p1["newfnames"] = [[item for item in row[1]["extitempaths"]]
# #                   for row in p1.iterrows()]
# print(p1.iloc[22]["sheets"])
# display(p1["nitems"].transpose())
# print(sorted([bname(item) for item in p1["extitempaths"]]))

display(pd.read_fwf(p1.iloc[44]["sheetpaths"][2])

[        category\tStim\tOldNumber\tRecognition_ACC\tRecognition_RESP  \
 0           New\tsporting_basketball_new.bmp\tNew14\t1\t2\t6460\t\t0   
 1                      New\tanimal_horse.bmp\tNew20\t1\t2\t2482\t\t0   
 2                New\tanimal_pigeon.bmp\tNew32\t0\t1\t2989\t6\t13492   
 3                    OLD\tfood_candy.bmp\tOld16\t1\t1\t2156\t6\t1997   
 4            New\tanimal_dolphin_old.bmp\tNew09\t1\t2\t3068\t6\t1997   
 5       OLD\tsporting_footballhelmet.bmp\tOld57\t1\t1\t1833\t6\t1519   
 6         New\tsporting_weighlifting.bmp\tNew25\t1\t2\t2001\t6\t1519   
 7               OLD\tvegie_garlic01a.bmp\tOld74\t1\t1\t1495\t8\t2435   
 8                   OLD\tvegie_radish.bmp\tOld77\t1\t1\t5056\t5\t590   
 9              OLD\tanimal_zebra_old.bmp\tOld13\t1\t1\t2285\t5\t1321   
 10              OLD\tanimal_penguin.bmp\tOld10\t1\t1\t3747\t9\t10853   
 11            New\tmusical_bagpipes.bmp\tNew35\t1\t2\t3412\t9\t10853   
 12      New\tkitchen_baking spatula.bmp\tNew01\t1\

### That's a clue!
 - There's a whole directory of files amon the files extracted

In [78]:
help(chr)


Help on built-in function chr in module builtins:

chr(i, /)
    Return a Unicode string of one character with ordinal i; 0 <= i <= 0x10ffff.



## Hint #1
### ValueError: invalid literal for int() with base 10

#### The number of bytes representing a character isn't what Python expected!
    - Files are possibly locked, broken or...
    - File was made with different encoding parameters
        - aka NOT 'utf-8'
###### How to find out next


In [73]:
#!/usr/bin/env python
# encoding: utf-8

def NEW_get_allsheetsdf(indir="~/extracted_eprime4"):
    
    '''Generate DataFrame indexing participants and their respective files'''
    indir = xpu(indir)
    p1 = pd.read_csv(join(indir, "participants.tsv"),
                     sep='\t').set_index("sub-ID")
    allsheetsdf = df([item for item in loadimages(row[1]['extdirpaths'])
                     if item.endswith(".txt")] for row in p1.iterrows())
    allsheets = flatten([[item for item in ls(row[1]['extdirpaths']) 
                          if os.path.splitext(item)[1] == ".txt"\
                         and "PRATIQUE" or "MACOSX" not in item]
                         for row in p1.iterrows()])
    p1['sheets'] = [[item for item in allsheets if row[0] in item]
                    for row in p1.iterrows()]
    

#     allsheetsdf = p1[["extitempaths"]]
#     allsheetsdf['prefix'] = [bname(row[1]['extitempaths']).split(\
#                                 row[0][4:4+len(row[0].split("sub-")[1])])[0]
#                              for row in allsheetsdf.iterrows()]
#     results = []
#     for row in allsheetsdf.iterrows():
#         detector = UniversalDetector()
#         detector.reset()
#         for line in open(row[1]['sheetpath'], 'rb'):
#             detector.feed(line)
#             if detector.done: break
#         detector.close()
#         results.append(pd.Series(detector.result, name=row[0]).squeeze())
#     rezz = df(results)
#     rezzndf = pd.concat([allsheetsdf, rezz], axis=1, sort=False)
    return p1

# def main():
#     allsheetsdf = NEW_get_allsheetsdf()

# if __name__ == '__main__':
#     main()
# p1 = NEW_get_allsheetsdf()
# print(p1['sheets'].iloc[22])
p1 = NEW_get_allsheetsdf()
display(p1['extitempaths'].iloc[0])

"['/home/fnadeau/extracted_eprime2/sub-3025432/3025432_658178_Tache_IRM', '/home/fnadeau/extracted_eprime2/sub-3025432/.sub-3025432_archives', '/home/fnadeau/extracted_eprime2/sub-3025432/__MACOSX']"

In [74]:
allsheetsdf = NEW_get_allsheetsdf()
# display(allsheetsdf.iloc[4]['prefix'])
# dirdict = df(((row[0], [item for item in row[1]['sheetpath'] if row[0] in item])
#                 for row in allsheetsdf.iterrows()))
# [(os.makedirs(join(maindir)))]
# print(allsheetsdf['sheetpath'].iloc[2])
# print(type(testval)==pd.core.frame.DataFrame)2
prefixes = sorted(allsheetsdf.prefix.unique().tolist())
print(len(prefixes))
print(prefixes)
len(allsheetsdf)
display(allsheetsdf)

AttributeError: 'DataFrame' object has no attribute 'prefix'

## Participant sheets cleanup procedure

#### Invalid particinant sheets have asc protected headers with a length exceeding 32767 characters

### 1- Remove headers right from the importation step 

In [77]:
help(list.count)
dir(list)

Help on method_descriptor:

count(...)
    L.count(value) -> integer -- return number of occurrences of value



['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

### To Remember

 - 78 images & 39 control (gray square) stimuli per trial
 - Thus, nTrials = nStim x 1.5
 - 103 valid participants


In [49]:
    # Initiate enpty list for each outputed variable for backup
    full_on_eve_enc, retsheets, fixnstimtimes, timings = [], [], [], []
    fixsheets, encsheets, s_ids, yallofems = [], [], [], []
    print(full_on_eve_enc, retsheets, fixnstimtimes)

[] [] []


In [78]:
p1 = pd.read_csv("~/extracted_eprime_AS_FWF/participants.tsv",
                 sep='\t').set_index("sub-ID")[["sheetpaths"]]

# test = [p1.iloc[33]['sheetpaths'].count(pre[1]) for pre in prefixes]

test = df(())


# test = [[row[0]['sheetpaths'].count(pre[1]) for pre in prefixes]
#         for row in p1.iterrows()]             
            # test = df(((row
#             for pre in enumerate(prefixes)))
# display(test)
print(len(p1.iloc[33]['sheetpaths']))

299


In [34]:
# -*- coding: ISO-8859-1 -*-

def NEW_prettyevents(participants_sheet_path="~/extracted_eprime2/participants.tsv",
                    outdir="~/cimaq_newsheets"):
# TrialNum ImageID Trial_part  onsetSec  durationSec
    maindir = xpu(dname(participants_sheet_path))
    
# Clear previous attempts
    p1 = pd.read_csv("~/extracted_eprime_AS_FWF/participants.tsv",
                     sep='\t').set_index("sub-ID")[["sheetpaths"]]
    [shutil.rmtree(join(outdir, row[0])) for row in p1.iterrows()]

# Headers for each of the 3 '.txt' files (open-source '.edat2' (E-prime) equivalent)
    prefixes = ["Onset-Event-Encoding_CIMAQ_", "Output-Responses-Encoding_CIMAQ_",
                "Output_Retrieval_CIMAQ_"]
    EncOnsetCols = ["TrialNum", "Condition", "TrialNum_perCondi",
                  "ImageID", "Trial_part", "onsetSec", "durationSec"]

    allsheets = NEW_get_allsheetsdf()

    # Initiate empty list for each outputed variable for backup
    full_on_eve_enc, retsheets, fixnstimtimes, timings = [], [], [], []
    fixsheets, encsheets, s_ids, yallofems = [], [], [], []
    
    # Create output directories structure
    allsheets["enc_outpaths"] = [join(outdir, row[0], "events") for row in allsheetsdf.iterrows()]
    allsheets["ret_outpaths"] = [join(outdir, row[0], "behavioral") for row in allsheetsdf.iterrows()]
    [(os.makedirs(join(outdir, row[0], "events"), exist_ok=True),
     os.makedirs(join(outdir, row[0], "behavioral"), exist_ok=True))
     for row in allsheetsdf.iterrows()]
    def sheetspersub(allsheets, prefixes):
        return dict((pre[1], [[(row[0], row[1]['sheetpath'])
                                for row in allsheets.iterrows()
                                if row[1]['prefix'] == prefixes[pre[0]]]
                               for row in p1.iterrows()])
                     for pre in in enumerate(prefixes))
    encsheetsA = [(row[0], row[1]['sheetpath'])
                  for row in allsheets.iterrows()
                  if row[1]['prefix'] == prefixes[0]]
        
    encsheetsB = [row[1]['sheetpath'] for row in allsheets.iterrows() if row[1]['prefix'] == prefixes[1]]
        
    retsheets = [row[1]['sheetpath'] for row in allsheets.iterrows()if row[1]['prefix'] == prefixes[2]]

    # Rows are individual sheets, 3 per participant
    for row in allsheetsdf.iterrows():
        if row[1]['prefix'] == prefixes[0]:

            newsheet = pd.read_fwf(row[1]['sheetpath'], encoding=row[1]['encoding'],
                header=None, sep='\t', names=EncOnsetCols).iloc[6:]

            # Identify stimulus & fixation cross as 2 possible conditions of a same trial
            # instead of separate (double) trials
            stimids = newsheet[["ImageID",
                               "TrialNum_perCondi"]].drop_duplicates(\
                               subset=["ImageID", "TrialNum_perCondi"]).reset_index(drop=True)
            s_ids.append((row[0]+"_"+row[1]['prefix']+"_stimids", stimids))
            newsheet.to_excel(join(maindir, "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))
            newsheet = pd.read_excel(join(\
                           maindir, "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx')).drop(\
                               ['TrialNum_perCondi', 'Condition'], axis=1)

            tempsheet = newsheet[['TrialNum', 'Trial_part', 'onsetSec', 'durationSec']]

            # Extract and concatenate relevant info
            fixsheet = df([row[1] for row in tempsheet.iterrows()
                                  if row[1]['Trial_part'] == 'Fixation'])
            timing = tempsheet.loc[[row[0] for row in tempsheet.iterrows()
                                     if row[0] not in fixsheet.index]]
            fixsheet = fixsheet.rename(columns={"onsetSec": "fixOnsetSec",
                                               "durationSec": "fixDurSec"})
            fixsheet = fixsheet.transpose().iloc[-2:].transpose(\
                           ).reset_index(drop=True)
#             fixsheets.append(((row[0]+"_"+row[1]['prefix']+"_fixsheet_", fixsheet)))

            timing = timing.rename(columns={"onsetSec": "stimOnsetSec",
                                            "durationSec": "stimDurSec"})
            timing = timing.transpose().iloc[-2:].transpose().reset_index(drop=True)
#             timings.append((row[0]+"_"+row[1]['prefix']+"_timing_", timing))
            fixnstimtime = pd.concat([timing, fixsheet], axis=1, sort=False)
#             fixnstimtimes.append((row[0]+"_"+row[1]['prefix']+"_fixnstimtime_", fixnstimtime))
            allofem = pd.concat([fixnstimtime, stimids], axis=1, sort=False)
            yallnametuple = (row[0]+"_"+row[1]['prefix'], allofem)
            yallofems.append(yallnametuple)
            row[1]['oldname'] = yallnametuple[0]

        if row[1]['prefix'] == prefixes[1]:
            encsheet = pd.read_csv(row[1]['sheetpath'],
                                   encoding=row[1]['encoding'],
                                   header=0,
                                   sep='\t').iloc[3:].fillna(False).rename(\
                           columns={"TrialNumber": "TrialNum",
                                    "Category": "Condition"}).set_index("TrialNum")
            encsheet = encsheet.drop(["TrialCode"], axis=1)
            encsheet["Condition"] = encsheet["Condition"].astype(\
                                        'str').replace({'CTL': '0', 'Enc': '1'})
            encnametuple = (row[0]+"_"+row[1]['prefix'],
                              encsheet.reset_index(drop=True))
            encsheets.append(encnametuple)
            row[1]['oldname'] = encnametuple[0]

        # Immediately removing last row since it was an Eprime error (St-Laurent, 2019)
        if row[1]['prefix'] == prefixes[2]:
            retsheet = pd.read_csv(row[1]['sheetpath'],
                                   encoding=row[1]['encoding'],
                                   header=0, sep='\t').iloc[:, :-1]
            retnametuple = (row[1]['prefix']+"_"+row[0],
                              retsheet.reset_index(drop=True))
            retsheets.append(retnametuple)
            row[1]['oldname'] = retnametuple[0]
        encoding = tuple(zip(sorted(yallofems), sorted(encsheets)))
        fullencsheets = [(item[0][0], pd.concat([item[0][1], item[1][1]], axis=1))
                        for item in encoding]
            
    return sorted(yallofems), sorted(encsheets), sorted(retsheets)


#### Previously displayed DataFrame facilitates participant-wise identification of files 
#### Here, each participant is indexed in allsheets as row[0] (index) according to
#### participant ID & sheet name

### Note

#### 1st line in code changes encoding for the wole script
#### More sheets showed up correctly with the following encoding mode versus UTF-8
#### Using the DataFrame displayed 2nd above cell makes it easier to order each file,
#### according to participant ID or any other relevant value
##### - (e.g. BIDS standard notation)


In [38]:
fullencsheets, retsheets = NEW_prettyevents()
print(len(yallofems), len(encsheets), len(retsheets))
# display(yallofems[0][0], yallofems[0][1].head(), encsheets[0][0], encsheets[0][1].head())
display(fullencsheets[0], retsheets[0])


102 103 101


('sub-3025432_Onset-Event-Encoding_CIMAQ_',
     stimOnsetSec stimDurSec fixOnsetSec fixDurSec ImageID TrialNum_perCondi  \
 0         11.542          3      14.551       5.5     NaN             CTL01   
 1         20.051          3       23.06       0.5   Old60             Enc01   
 2         23.561          3       26.57         1   Old01             Enc02   
 3         27.557          3      30.566         1   Old24             Enc03   
 4         31.569          3      34.578       1.5   Old68             Enc04   
 ..           ...        ...         ...       ...     ...               ...   
 112       694.41          3     697.419         6   Old45             Enc75   
 113       703.42          3     706.429        10     NaN             CTL39   
 114      716.927          3     719.936         5   Old18             Enc76   
 115      724.934          3     727.943         1   Old33             Enc77   
 116      728.929          3     731.938        18   Old14             Enc78

('Output_Retrieval_CIMAQ__sub-3025432',
     category                          Stim OldNumber  Recognition_ACC  \
 0        OLD            animal_penguin.bmp     Old10                1   
 1        OLD      sporting_bicycle_old.bmp     Old55                0   
 2        OLD           fruit_lemon_new.bmp     Old69                1   
 3        New            vegie_broccoli.bmp     New36                1   
 4        OLD          animal_zebra_old.bmp     Old13                1   
 ..       ...                           ...       ...              ...   
 112      New       kitchen_spatula_old.bmp     New37                0   
 113      OLD        musical_violin_old.bmp     Old51                1   
 114      OLD              vegie_carrot.bmp     Old78                1   
 115      OLD  sporting_karate clothing.bmp     Old60                1   
 116      OLD        fruit_green grapes.bmp     Old68                0   
 
      Recognition_RESP  Recognition_RT  Spatial_RESP  Spatial_RT  
 0 

In [ ]:
# Iterating over allsheets allows customized decoding parameters for each subject or sheet
# Note: Files containing this prefix have twice the number of trials than other sheets
# due to fixation cross and stimuli presentation stages of a same trial being coded as
# whole seperate trials. The following code decodes and formats each sheet properly.
# Passing the headers defined above directly as arguments while loading sheets
# has proven very uneffective. The 'sniff_encodtype' function above help, but several
# unkown properties/values/fields/dimensions/etc remain to assert.

        # Decoding data 'as is' and cropping a priori irrelevant/redundant fields, saving to xlsx,
        # reopening, editing, then saving to desired format has proven to be the most efficient strategy, meaning
        # *no sheets were harmed during the making of this script.*
        
EncOutCols = ["stim_category", "stim_name", "recognition_accuracy",
              "recognition_responsetime", "position_response",
              "position_responsetime"]



RetCols={
            'category': 'old_new', 'Stim': 'stim_file',
            'OldNumber': 'stim_id', 'Recognition_ACC': 'recognition_accuracy',
            'Recognition_RESP': 'recognition_response',
            'Recognition_RT': 'recognition_responsetime',
            'Spatial_RESP': 'position_response',
            'Spatial_RT': 'position_responsetime',
            'Spatial_ACC(à corriger voir output-encodage)': 'position_accuracy'
        }

In [61]:
# -*- coding: ISO-8859-1 -*-
#             newsheet['Condition'] = newsheet['Condition'].astype('str').replace({'CTL': '0', 'Enc': '1'})
#  ('sub-5477234_Onset-Event-Encoding_CIMAQ_',
#  ('sub-5477234_Output-Responses-Encoding_CIMAQ_',
#  ('sub-5477234_Output_Retrieval_CIMAQ_',
#  ('sub-6417837_Onset-Event-Encoding_CIMAQ_',
#  ('sub-6417837_Output-Responses-Encoding_CIMAQ_',
#  ('sub-6417837_Output_Retrieval_CIMAQ_',
#  ('sub-7674650_Onset-Event-Encoding_CIMAQ_',
#  ('sub-7674650_Output_Retrieval_CIMAQ_'.,
#  ('sub-7707569_._Onset-Event-Encoding_CIMAQ_',
#  ('sub-7707569_._Output-Responses-Encoding_CIMAQ_',
#  ('sub-7707569_._Output_Retrieval_CIMAQ_',

def NEW_prettyevents(participants_sheet_path="~/extracted_eprime_AS_FWF/participants.tsv"):
    maindir = xpu(dname(participants_sheet_path))

    EncOutCols = ["stim_category", "stim_name", "recognition_accuracy", "recognition_responsetime",
                      "position_response", "position_responsetime"]

    EncOnsetCols = ["TrialNum", "Condition", "TrialNum_perCondi",
                      "ImageID", "Trial_part", "onsetSec", "durationSec"]
    RetCols={
            'category': 'old_new',
            'Stim': 'stim_file',
            'OldNumber': 'stim_id',
            'Recognition_ACC': 'recognition_accuracy',
            'Recognition_RESP': 'recognition_response',
            'Recognition_RT': 'recognition_responsetime',
            'Spatial_RESP': 'position_response',
            'Spatial_RT': 'position_responsetime',
            'Spatial_ACC(à corriger voir output-encodage)': 'position_accuracy'
            }
    prefixes = ["Onset-Event-Encoding_CIMAQ_", "Output-Responses-Encoding_CIMAQ_",
                "Output_Retrieval_CIMAQ_"]

    sheets = get_allsheets()
    rezz = sniff_encodtype(sheets)
    allsheets = pd.concat([sheets, rezz], axis=1)
    newsheets = []
    part1 = []
    part2 = []
    encsheets = []
    for row in allsheets.iterrows():
        if row[1]['prefix'] == prefixes[0]:
            newsheet = pd.read_fwf(\
                row[1]['sheetpath'], encoding=row[1]['encoding'],
                header=None, sep='\t',
                names=EncOnsetCols).iloc[6:].drop(columns=['TrialNum_perCondi',
                                                           'Condition'])
            newsheet.to_excel(join(maindir, "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))
            newsheet = pd.read_excel(join(\
                            maindir,
                            "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))
            
            tempsheet = newsheet[['TrialNum', 'Trial_part',
                                  'onsetSec',  'durationSec']]
            fixsheet = df([row[1] for row in tempsheet.iterrows()
                                  if row[1]['Trial_part'] == 'Fixation'])
            timings = tempsheet.loc[[row[0] for row in tempsheet.iterrows()
                                     if row[0] not in fixsheet.index]]
            fixsheet = fixsheet.rename(columns={"onsetSec": "fixOnsetSec",
                                               "durationSec": "fixDurSec"})
            fixsheet = fixsheet.transpose().iloc[-2:].transpose(\
                           ).reset_index().drop(['index'], axis=1)
            fixsheet = fixsheet.reset_index()
            part1.append((row[0]+"_"+"fixsheet", fixsheet))

            timings = timings.rename(\
                          columns={"onsetSec": "stimOnsetSec",
                                   "durationSec": "stimDurSec"}).transpose(\
                                       ).iloc[-2:].transpose().reset_index().drop(['index'],
                                                                                  axis=1)
            part2.append((row[0]+"_"+row[1]['prefix']+"_stimsheet_", timings))
            encsheets.append((row[0]+"_"+row[1]['prefix']+"encsheeta", timings.join(fixsheet, on=[timings.index])))
    return encsheets
#         if row[1]['prefix'] == prefixes[0]:
#             newsheet = pd.read_fwf(\
#                 row[1]['sheetpath'], encoding=row[1]['encoding'],
#                 header=None, sep='\t',
#                 names=EncOnsetCols).iloc[6:].drop(columns=['TrialNum_perCondi',
#                                                            'Condition'])
#             newsheet.to_excel(join(maindir, "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))
#             tempsheet = pd.read_excel(join(\
#                             maindir,
#                             "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))[['TrialNum',
#                                                                            'Trial_part',
#                                                                             'onsetSec',
#                                                                             'durationSec']]


#         if row[1]['prefix'] == prefixes[1]:
#             newsheet = pd.read_csv(row[1]['sheetpath'],
#                                    encoding=row[1]['encoding'],
#                                    header=0, sep='\t').iloc[3:].fillna(False).rename(\
#                                        columns={"TrialNumber": "TrialNum",
#                                                 "Category": "Condition"}).set_index("TrialNum")
            
#         if row[1]['prefix'] == prefixes[1]:
#             newsheet = pd.read_csv(row[1]['sheetpath'],
#                                    encoding=row[1]['encoding'],
#                                    header=0, sep='\t').iloc[3:].fillna(False).rename(\
#                                        columns={"TrialNumber": "TrialNum",
#                                                 "Category": "Condition"}).set_index("TrialNum")
#             newsheet["Condition"] = newsheet["Condition"].astype('str').replace({'CTL': '0', 'Enc': '1'})
#             encsheets.append((row[0]+"_"+row[1]['prefix'], newsheet.reset_index()))
            
#             newsheet["Condition"] = newsheet["Condition"].astype('str').replace({'CTL': '0', 'Enc': '1'})
#             encsheets.append((row[0]+"_"+row[1]['prefix'], newsheet))

#         if row[1]['prefix'] == prefixes[2]:
#             newsheet = pd.read_csv(row[1]['sheetpath'],
#                                    encoding=row[1]['encoding'],
#                                    header=0, sep='\t').iloc[:, :-1]
#             retsheets.append((row[0]+"_"+row[1]['prefix'], newsheet))
    
#     tempenc = tuple(zip(sorted(part1), sorted(part2)))
#     encmain = [(row[0]+"_task-memory_events.tsv", item[0][1].append(item[1][1])) for item in tempenc]
#     encsheets.append(encmain)
#     [encsheet[1].to_csv(join(maindir, encsheet[0]), sep='\t') for encsheet in encsheets]
#     [newsheet[1].to_csv(join(maindir, newsheet[0]), sep='\t') for newsheet in newsheets]



#         else:
#             newsheet = pd.read_fwf(row[1]['sheetpath'],
#                                    encoding=row[1]['encoding'],
#                                    header=True, index=True, sep='\t')


#         newsheets.append((row[0]+"_"+row[1]['prefix'], newsheet))
#     [[pd.merge(fix)]]
#     encsheets = [pd.merge(part[0], part[1]) for part in tuple(zip(sorted(part1), sorted(part2)))]
#     return newsheets, tempenc

            
        
#         if row[1]['prefix'] == prefixes[0]:
#             newsheet = pd.read_fwf(\
#                 row[1]['sheetpath'], encoding=row[1]['encoding'],
#                 header=None, sep='\t',
#                 names=EncOnsetCols).iloc[6:].drop(columns=['TrialNum_perCondi',
#                                                            'Condition'])
#             newsheet.to_excel(join(maindir, "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))
#             newsheet = pd.read_excel(join(\
#             encsheets.append(newsheet['ImageID'].dropna())
#             tempsheet = newsheet[['TrialNum', 'Trial_part',
#                                   'onsetSec', 
#                                   'durationSec']]
#             fixsheet = df([row[1] for row in tempsheet.iterrows()
#                             maindir,
#                             "temp_"+row[0]+"_"+row[1]['prefix']+'.xlsx'))
#                                   if row[1]['Trial_part'] == 'Fixation'])
#             timings = tempsheet.loc[[row[0] for row in tempsheet.iterrows()
#                                      if row[0] not in fixsheet.index]]
#             fixsheet = fixsheet.rename(columns={"onsetSec": "fixOnsetSec",
#                                                "durationSec": "fixDurSec"}).transpose().iloc[-2:].transpose().reset_index().drop(['index'], axis=1)
#             part1.append((row[0]+"_"+"fixsheet", fixsheet.reindex()))
            
#             timings = timings.rename(columns={"onsetSec": "stimOnsetSec",
#                                               "durationSec": "stimDurSec"}).transpose(\
#                                                   ).iloc[-2:].transpose().reset_index(\
#                                                       ).drop(['index'], axis=1)
#             part2.append((row[0]+"_"+"stimsheet", timings))



# encsheets = NEW_prettyevents()
# uniformity = [sheet[1].shape for sheet in newsheets]
# summary = sorted(list(dict.fromkeys(uniformity)))
# display([sheet[1].shape for sheet in newsheets])
print(len(encsheets))

102


In [17]:
#!/usr/bin/env python
# encoding: utf-8

import chardet
import csv
import json
import os
import re
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from os.path import splitext
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages
from NEW_extract_events2 import NEW_extract_events2
from NEW_get_allsheets import NEW_get_allsheets
from chardet.universaldetector import UniversalDetector

def NEW_get_allsheets(participants_sheet_path="~/extracted_eprime_AS_FWF/participants.tsv"):
    '''Generate DataFrame indexing participants and their respective files'''
    maindir = xpu(dname(participants_sheet_path))
    p1 = pd.read_csv(join(maindir, bname(participants_sheet_path)),
                     sep='\t').set_index("sub-ID")[["sheetpaths"]]

    allsheets = sorted(flatten([[join(maindir, row[0], file)
                                 for file in ls(join(maindir, row[0]))
                                 if file.endswith(".txt") and \
                                  bool(file.find("MACOSX") != -1)==False]
                                for row in p1.iterrows()]))
    
    allsheets = sorted([(bname(dname(sheet)), sheet) for sheet in allsheets])
    allsheetsdf = df(allsheets, columns=['sub-ID', 'sheetpath']).set_index('sub-ID')
    allsheetsdf['prefix'] = [bname(row[1]['sheetpath']).split(\
                                row[0][4:4+len(row[0].split("sub-")[1])])[0]
                             for row in allsheetsdf.iterrows()]
    allsheetsdf.to_json(join(dname(participants_sheet_path), "allsheetsdf.json"), indent=4)

    return allsheetsdf

def main():
    allsheetsdf = NEW_get_allsheets()

if __name__ == '__main__':
    main()
    

ImportError: cannot import name 'NEW_get_allsheets'

In [47]:
display(encsheets)

[('sub-3025432_Output-Responses-Encoding_CIMAQ___encsheet_',
      Condition TrialCode OldNumber  CorrectSource Stim_RESP  Stim_RT  Stim_ACC
  0           0     CTL01     False              8         1     1402         0
  1           1     Enc01     Old60              6     False        0         1
  2           1     Enc02     Old01              5         1     1822         0
  3           1     Enc03     Old24              6         1     1930         0
  4           1     Enc04     Old68              9         1     1814         0
  ..        ...       ...       ...            ...       ...      ...       ...
  112         1     Enc75     Old45              6         1     1974         0
  113         0     CTL39     False              9         1      836         0
  114         1     Enc76     Old18              5         1     1201         0
  115         1     Enc77     Old33              5         1     1249         0
  116         1     Enc78     Old14              8         

In [ ]:
#             part1_encfix = pd.merge([newsheet['ImageID'].values, fixsheet, encsheet])
#             tempsheet['Trial_part'] = tempsheet['Trial_part'].replace({})
#             encsheet = tempsheet["Trial_part", "onsetSec", "durationSec"]
#                                  for row in tempsheet.iterrows()
#                                  if row[1]['Trial_part'='Encoding'].drop_duplicates(subset=["Trial_part",
#                                                                                              "onsetSec",
#                                                                                            "durationSec"])
#             fixsheet = tempsheet[row[1]["Trial_part", "onsetSec", "durationSec"]
#                                  for row in tempsheet.iterrows()
#                                  if row[1]['Trial_part'='Fixation']].drop_duplicates(subset=[encsheet.columns])


#             tempsheet["Trial_part"] = tempsheet["Trial_part"].astype('str').replace({'CTL': '0', 'Enc': '1'})
#             fixsheet = newsheet[["Trial_part", "onsetSec",
#                                  "durationSec"]].set_index(\
#                                      "Trial_part").iloc[7:].transpose().squeeze()
#             fixsheet = tempsheet[["Condition", 'onsetSec', 'durationSec']].iloc[[row[0]\
#                            for row in tempsheet.iterrows()
#                            if row[1]['Trial_part'] == 'Fixation']].rename(\
#                                columns={"onsetSec": "fix_onset", "durationSec": "fix_duration_sec"}).drop_duplicates()
#             tempsheet = tempsheet.drop(["Trial_part", "onsetSec", "durationSec"], axis=1).reindex_like(fixsheet)
#             correctsheet = pd.merge(tempsheet, fixsheet)
# #            tempsheet["fix_onset"], tempsheet["fix_duration= fixsheet["fix_onset"], fixsheet["fix_duration"]
# #           oksheet = tempsheet.join((pd.Series(item[1], name=item[0]) for item in fixsheet.iteritems()))
# #          tempsheet = newsheet.drop(["Trial_part", "onsetSec", "durationS"])
# #         tempsheet["stim_onset"] = [newsheetdf.drop_duplicates(subset=['onsetSec'])["onsetSec"]]
# #        tempsheet["stim_duration"] = [newsheetdf.drop_duplicates(subset=['durationSec'])["durationSec"]]

#             encsheet = df(newsheet.iloc[[row[0] for row in newsheet.iterrows()
#                                        if row[1]['Trial_part'] != 'Fixation']])
#             part1.append((row[0]+"_"+row[1]['prefix'], part1_encfix))
#             newsheet.to_csv(join(maindir, 'tempsheet_'+row[0]+"_"+row[1]['prefix']+'.csv'), sep=' ')
#             newsheet = pd.read_fwf(join(maindir, 'tempsheet_'+row[0]+"_"+row[1]['prefix']+'.csv'),
#                                    widths=[9, 18, 9, 12, 10, 11])
#         tempsheets = df(((bname(sheet), join(maindir, sheet)) for sheet in ls(maindir)
#                           if 'tempsheet' in sheet), columns=['sub-ID', 'sheetpath'])
#         test = sniff_encodtype(allsheetsdf=tempsheets)
#         temp = pd.concat([tempsheets, test], axis=1)

(0      onsetSec       11.542
 durationSec       3.0
 Na...
 1      onsetSec       20.051
 durationSec       3.0
 Na...
 2      onsetSec       23.561
 durationSec       3.0
 Na...
 3      onsetSec       27.557
 durationSec       3.0
 Na...
 4      onsetSec       31.569
 durationSec       3.0
 Na...
                              ...                        
 112    onsetSec       694.41
 durationSec       3.0
 Na...
 113    onsetSec       703.42
 durationSec       3.0
 Na...
 114    onsetSec       716.927
 durationSec        3.0
 ...
 115    onsetSec       724.934
 durationSec        3.0
 ...
 116    onsetSec       728.929
 durationSec        3.0
 ...
 Length: 117, dtype: object,
 0      onsetSec       14.551
 durationSec     5.500
 Na...
 1      onsetSec       23.06
 durationSec     0.50
 Name...
 2      onsetSec       26.57
 durationSec     1.00
 Name...
 3      onsetSec       30.566
 durationSec     1.000
 Na...
 4      onsetSec       34.578
 durationSec     1.500
 Na...
             

In [182]:
###########################################################################################
#             if len(newsheet[1].values) > 32727:
#                 newsheet[1].to_csv(join(maindir, sheet[0]+"_"+"BROKEN".csv))
#             if newsheet[1].shape[1] == 1:
#                 newsheet2 = df((str(row[1].values).split('\t')
#                                 for row in newsheet[1].iterrows()))
#                 newsheet = (sheet[0], newsheet2)
                
#             newsheet[1].to_csv(join(maindir, sheet[0], sheet[0]+"_"+"OnsetEventEncodingCIMAQ.tsv"), sep='\t')
#             newsheet = ((sheet[0]+"_"+"OnsetEventEncodingCIMAQ.tsv",
#                             pd.read_csv(join(maindir, sheet[0],
#                                         sheet[0]+"_"+"OnsetEventEncodingCIMAQ.tsv"), sep='\t')))
############################################################################################
# #             temp = newsheet.loc
# #             newsheets.append(join(dname(sheet), "OutputRetrievalCIMAQ.csv"), newsheet)
# #             if "Spatial_ACC(à", 
# #             if newsheet.shape[0] == 120:
# #                 newsheet.drop(index=[1, 2 , 3], inplace=True)
#         if "Output-Responses-Encoding_CIMAQ_" in sheet:
#             newsheet = pd.read_fwf(sheet, encoding='ISO-8859-1', header=None,
#                                    index_col=0, dtype='object', convert_float=False)
#             newsheet.to_csv(join(dname(sheet), "OutputResponsesEncodingCIMAQ.csv"))
#             newsheet = pd.read_csv(join(dname(sheet), "OutputResponsesEncodingCIMAQ.csv"))
#             newsheets.append((join(dname(sheet), "OutputRetrievalCIMAQ.csv"), newsheet)

#         if "Output_Retrieval_CIMAQ_" in sheet:
#             newsheet = pd.read_fwf(sheet, header=None, encoding='ISO-8859-1',
#                                    index_col=0, dtype='object')
#             newsheet.to_csv(join(dname(sheet), "OutputRetrievalCIMAQ.csv"))
#             newsheet = pd.read_csv(join(dname(sheet), "OutputRetrievalCIMAQ.csv"))
#             newsheets.append((join(dname(sheet), "OutputRetrievalCIMAQ.csv"), newsheet))
#             newsheets.append(newsheet)
#     infos = df(((newsheet[0], newsheet[1].shape, newsheet[1].head(),
#                                newsheet[1].index, newsheet[1].columns)
#                 for newsheet in newsheets), columns=["name", "shape", "header", "index", "columns"])
    
#     return newsheets, infos

345

In [ ]:
def removeComments(string):
    string = re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,string) # remove all occurrences streamed comments (/*COMMENT */) from string
    string = re.sub(re.compile("//.*?\n" ) ,"" ,string) # remove all occurrence single-line comments (//COMMENT\n ) from string
    return string

In [ ]:
############ join(dname(sheet), os.path.splitext(bname(sheet))[0]+".tsv")
############ 'Spatial_ACC(à corriger voir output-encodage)': 'position_accuracy'}
# '''
#  names={'TrialNumber': 'trial_number',
#         'Category': 'trial_type',
#         'OldNumber': 'stim_id',
#         'CorrectSource': 'position_correct',
#         'Stim_RESP': 'response',
#         'Stim_RT': 'response_time'}
# , sheet_name=join(bname(dname(sheet)), bname(sheet))
# 'Spatial_ACC(à corriger voir output-encodage)': 'position_accuracy'})
# lines <- readLines("g20095us.txt")

# non_ascii <- function(x) {
#   any(charToRaw(x) > 127)
# }

# bad <- vapply(lines, non_ascii, logical(1), USE.NAMES = FALSE)
# lines[bad]

In [4]:
# <pre>3025432_658178_Tache_IRM
# Onset-Event-Encoding_CIMAQ_3025432_session1B.txt
# Output-Responses-Encoding_CIMAQ_3025432_session1B.txt
# Output_Retrieval_CIMAQ_3025432_1.txt
# </pre>

def NEW_extract_events2(
    zipdir="~/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime",
    outdir="~/extracted_eprime"):
    # Extract to new directories
    outdir = xpu(outdir)
    zipdir = xpu(zipdir)
    participants = df([("sub-"+bname(zfile).split("_")[0],
                        bname(zfile).split("_")[0],
                        bname(zfile).split("_")[1], bname(zfile),
                        zfile, join(outdir, "sub-"+bname(zfile).split("_")[0]))
                       for zfile in loadimages(zipdir)
                       if zfile.endswith(".zip")],
                      columns=["sub-ID", "pscid", "dccid", "zfilename",
                               "zipdirpath",
                               "extdirpath"]).set_index("sub-ID").sort_index()
    # Extract files
    os.makedirs(outdir, exist_ok=True)
    for row in participants.iterrows():
        archv = zipfile.ZipFile(row[1]["zipdirpath"], 'r')
        archv.extractall(row[1]["extdirpath"])
        archv.close()
    [[shutil.move(file, join(outdir, row[0], os.path.splitext(bname(file))[0]+".tsv"))
#                              os.path.splitext(bname(file))[0]+".tsv"))
            for file in loadimages(row[1]["extdirpath"])
      if "CIMAQ" in bname(file)]
     for row in participants.iterrows()]
    # Clear empty archives
    for topdir, folders, files in os.walk(outdir, topdown=False):
        for folder in folders:
            if "__MACOSX" in join(topdir, folder):
                shutil.rmtree(join(topdir, folder))
    for row in participants.iterrows():
        row[1]["nflist"] = []
        for file in loadimages(join(outdir, row[0])):
            if file.endswith(".tsv") and "CIMAQ" in file:
                filedf = pd.read_csv(file, sep='\t', encoding='ISO-8859-1')
                filedf["strvals"] = [str(row[1]) for row in filedf.iterrows()]
                filedf["splitvals"] = [[str(row[1]).split()] for row in filedf.iterrows()]
                newfile = filedf[["strvals", "splitvals"]]
                row[1]["nflist"].append(newfile)
    return participants

    
        
#     [[(file, pd.read_csv(file, sep='\t'))
#            for file in loadimages(join(outdir, row[0])) if file.endswith(".csv") and "CIMAQ" in file])
#           for row in participants.iterrows()]
    




#     participants.to_csv(xpu("~/extracted_eprime/participants.tsv"), sep='\t')
#     print(participants)

def main():
    participants = NEW_extract_events2()
    display(participants)
if __name__ == '__main__':
    main()


,pscid,dccid,zfilename,zipdirpath,extdirpath
sub-ID,,,,,
sub-3025432,3025432,658178,3025432_658178_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-3025432
sub-3123186,3123186,920577,3123186_920577_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-3123186
sub-3163875,3163875,199801,3163875_199801_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-3163875
sub-3291977,3291977,748676,3291977_748676_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-3291977
sub-3303819,3303819,893639,3303819_893639_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-3303819
...,...,...,...,...,...
sub-9003010,9003010,785245,9003010_785245_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-9003010
sub-9249304,9249304,778749,9249304_778749_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-9249304
sub-9296157,9296157,955548,9296157_955548_Tache_IRM.zip,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-9296157


In [2]:
print("9249304_778749_Tache_IRM,zip".split("_"))

['9249304', '778749', 'Tache', 'IRM,zip']


sub-ID
sub302543    [(/home/fnadeau/extracted_eprime/sub-302543_ex...
sub312318    [(/home/fnadeau/extracted_eprime/sub-312318_ex...
sub316387    [(/home/fnadeau/extracted_eprime/sub-316387_ex...
sub329197    [(/home/fnadeau/extracted_eprime/sub-329197_ex...
sub330381    [(/home/fnadeau/extracted_eprime/sub-330381_ex...
                                   ...                        
sub900301    [(/home/fnadeau/extracted_eprime/sub-900301_ex...
sub924930    [(/home/fnadeau/extracted_eprime/sub-924930_ex...
sub929615    [(/home/fnadeau/extracted_eprime/sub-929615_ex...
sub949407    [(/home/fnadeau/extracted_eprime/sub-949407_ex...
sub992916    [(/home/fnadeau/extracted_eprime/sub-992916_ex...
Name: clean_mover, Length: 103, dtype: object

,zfilename,pscid,dccid,zipdirpath,extdirpath,extfilepaths,clean_mover
sub-ID,,,,,,,
sub302543,3025432_658178_Tache_IRM.zip,3025432,658178,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-302543_ext_...,[/home/fnadeau/extracted_eprime/sub-302543_ext...,[(/home/fnadeau/extracted_eprime/sub-302543_ex...
sub312318,3123186_920577_Tache_IRM.zip,3123186,920577,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-312318_ext_...,[/home/fnadeau/extracted_eprime/sub-312318_ext...,[(/home/fnadeau/extracted_eprime/sub-312318_ex...
sub316387,3163875_199801_Tache_IRM.zip,3163875,199801,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-316387_ext_...,[/home/fnadeau/extracted_eprime/sub-316387_ext...,[(/home/fnadeau/extracted_eprime/sub-316387_ex...
sub329197,3291977_748676_Tache_IRM.zip,3291977,748676,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-329197_ext_...,[/home/fnadeau/extracted_eprime/sub-329197_ext...,[(/home/fnadeau/extracted_eprime/sub-329197_ex...
sub330381,3303819_893639_Tache_IRM.zip,3303819,893639,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-330381_ext_...,[/home/fnadeau/extracted_eprime/sub-330381_ext...,[(/home/fnadeau/extracted_eprime/sub-330381_ex...
...,...,...,...,...,...,...,...
sub900301,9003010_785245_Tache_IRM.zip,9003010,785245,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-900301_ext_...,[/home/fnadeau/extracted_eprime/sub-900301_ext...,[(/home/fnadeau/extracted_eprime/sub-900301_ex...
sub924930,9249304_778749_Tache_IRM.zip,9249304,778749,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-924930_ext_...,[/home/fnadeau/extracted_eprime/sub-924930_ext...,[(/home/fnadeau/extracted_eprime/sub-924930_ex...
sub929615,9296157_955548_Tache_IRM.zip,9296157,955548,/home/fnadeau/../../data/cisl/DATA/cimaq_03-19...,/home/fnadeau/extracted_eprime/sub-929615_ext_...,[/home/fnadeau/extracted_eprime/sub-929615_ext...,[(/home/fnadeau/extracted_eprime/sub-929615_ex...


In [ ]:
#!/usr/bin/env python
# encoding: utf-8

import os
import re
import sys

import argparse
import glob
import logging
import numpy as np
import zipfile
from numpy import nan as NaN
import pandas as pd
import shutil
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages

In [6]:
display(participants["clean_mover"].tolist())

[[('/home/fnadeau/extracted_eprime/sub-302543_ext_3025432_658178_Tache_IRM.zip/3025432_658178_Tache_IRM/Retrieval-3025432-1.txt',
   '/home/fnadeau/sorted_extracted_eprime/mainfiles/sub302543/sub302543_Retrieval-3025432-1.txt'),
  ('/home/fnadeau/extracted_eprime/sub-302543_ext_3025432_658178_Tache_IRM.zip/3025432_658178_Tache_IRM/Retrieval-3025432-1.edat2',
   '/home/fnadeau/sorted_extracted_eprime/mainfiles/sub302543/sub302543_Retrieval-3025432-1.edat2'),
  ('/home/fnadeau/extracted_eprime/sub-302543_ext_3025432_658178_Tache_IRM.zip/3025432_658178_Tache_IRM/Output_Retrieval_CIMAQ_3025432_1.txt',
   '/home/fnadeau/sorted_extracted_eprime/mainfiles/sub302543/sub302543_Output_Retrieval_CIMAQ_3025432_1.txt'),
  ('/home/fnadeau/extracted_eprime/sub-302543_ext_3025432_658178_Tache_IRM.zip/3025432_658178_Tache_IRM/PRATIQUE-3025432-1.txt',
   '/home/fnadeau/sorted_extracted_eprime/mainfiles/sub302543/sub302543_PRATIQUE-3025432-1.txt'),
  ('/home/fnadeau/extracted_eprime/sub-302543_ext_302543

In [ ]:
test = [[("a","b"), ("c", "d")], [("e", "f"), ("g", "h")], [("l", "m"), ("n", "o")]]
flat_test = flatten(test)
display(flat_test)

In [ ]:
def extract_events(zipdir="~/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime"):
    # Extract to new directories
    zipdir = xpu(zipdir)

    participants = df([(bname(zfile), bname(zfile)[:7], bname(zfile)[8:14],
                        zfile, xpu(join("~/extrcted_eprime",
                                        "sub-"+bname(zfile)[:6]+"_ext_"+os.path.splitext(bname(zfile))[0])))
                       for zfile in loadimages(zipdir)
                       if zfile.endswith(".zip")],
                      columns=["zfilename", "pscid", "dccid",
                               "zipfpath", "extfpath"]).set_index("zfilename").sort_index()
    
    xtractor = tuple(zip(participants["zipfpath"], participants["extfpath"]))
    os.makedirs(xpu("~/extracted_eprime"), exist_ok=True)

    [[zipfile.ZipFile(item[0], 'r').extractall(item[1])]
     for item in xtractor]
    participants["extfilepaths"] = [[item for item in loadimages(item[1])
                                     if item.endswith(".txt")]
                                    for item in xtractor]
    return participants
participants = extract_events()

In [ ]:
display(participants["extfilepaths"].values)

In [ ]:

def get_arguments():
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description="",
        epilog="""
        Convert behavioural data from cimaq to bids format
        Input: Folder with zip files
        """)

    parser.add_argument(
        "-d", "--idir",
        required=True, nargs="+",
        help="Folder to be sorted")

    parser.add_argument(
        "-o", "--odir",
        required=True, nargs="+",
        help="Output folder - if doesn\'t exist it will be created.")

    parser.add_argument(
        '--log_level', default='INFO',
        choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
        help='Log level of the logging class.')

    args = parser.parse_args()
    if len(sys.argv) == 1:
        parser.print_help()
        sys.exit()
    else:
        return args


# def get_all_ids(iFolder):
#     """ List all ZipFile and get all IDs
#     Parameters:
#     ----------
#     iFolder: string (input folder)

#     Return:
#     ----------
#     ids: list of tuple (behavioral ID, IRM ID)
#     """
#     if not os.path.exists(iFolder):
#         sys.exit('This folder doesn\'t exist: {}'.format(iFolder))
#         return
#     ids = []
#     allZipFiles = glob.glob(join(iFolder, '*.zip'))
#     for currZipFile in allZipFiles:
#         currZipFile = bname(currZipFile)
#         ids.append((currZipFile.split('_')[0], currZipFile.split('_')[1]))

#     if not ids:
#         sys.exit('This folder doesn\'t contain any zip files')
#         return
#     else:
#         return ids


def set_subject_data(bID, iFolder, oFolder):
    """
    Parameters:
    ----------
    bID: string (PSCID used to identify participants during data collection)
    datadir: string (input folder)
    oFolder: string (output folder)

    Return:
    ----------
    sub_files: list (three input files)
    """
    logging.debug('Subject PSCID": {}'.format(bID))

    prefix = ['Output-Responses-Encoding_CIMAQ_*',
              'Onset-Event-Encoding_CIMAQ_*',
              'Output_Retrieval_CIMAQ_*']

    sub_files = []
    s_dir = glob.glob(join(iFolder, bID+'*IRM.zip'))

    if len(s_dir) != 1:
        logging.error(' Multiple directories match \
                       this subject PSCID: {}'.format(bID))
    else:
        s_path = join(oFolder, bID+'*')
        s_out = glob.glob(s_path)
        if not s_out:
            z_ref = zipfile.ZipFile(s_dir[0], 'r')
            z_ref.extractall(oFolder)
            z_ref.close()
            s_out = glob.glob(s_path)

        if len(s_out) == 1:
            s_out = s_out[0]
            for nPrefix in prefix:
                file = glob.glob(join(s_out, nPrefix))
                if len(file) == 1:
                    sub_files.append(file[0])
                else:
                    logging.error('Multiple files found'.format(bID))
        else:
            logging.error('Multiple folders found'.format(bID))

    return sub_files


def cleanMain(mainFile):
    """
    Parameters:
    ----------
    mainFile: pandas object

    Return:
    ----------
    mainFile: pandas object
    """
    # remove first three junk rows (blank trials): CTL0, Enc00 and ENc000
    mainFile.drop([0, 1, 2], axis=0, inplace=True)
    # re-label columns
    mainFile.rename(columns={'TrialNumber': 'trial_number',
                             'Category': 'trial_type',
                             'OldNumber': 'stim_id',
                             'CorrectSource': 'position_correct',
                             'Stim_RESP': 'response',
                             'Stim_RT': 'response_time'}, inplace=True)
    # remove redundant columns
    mainFile.drop(['TrialCode', 'Stim_ACC'], axis=1, inplace=True)
    # re-order columns
    cols = ['trial_number', 'trial_type', 'response', 'response_time',
            'stim_id', 'position_correct']
    mainFile = mainFile[cols]
    # change in-scan reaction time from ms to s
    mainFile['response_time'] = mainFile['response_time'].astype('float64',
                                                                 copy=False)
    mainFile['response_time'] = mainFile['response_time'].div(1000)
    # insert new columns
    colNames = ['onset', 'duration', 'offset', 'stim_file', 'stim_category',
                'stim_name', 'recognition_accuracy',
                'recognition_responsetime', 'position_response',
                'position_accuracy', 'position_responsetime']
    dtype = [NaN, NaN, NaN, 'None', 'None', 'None', -1, NaN, -1, -1, NaN]
    colIndex = [0, 1, 2, 8, 9, 10, 11, 12, 14, 15, 16]
    for i in range(0, 11):
        mainFile.insert(loc=colIndex[i],
                        column=colNames[i],
                        value=dtype[i],
                        allow_duplicates=True)
    return mainFile  # modified in-place


def cleanOnsets(onsets):
    """
    Description:
        Label columns and remove first six junk rows
        (3 junk trials; 2 rows per trial).

    Parameters:
    ----------
    onsets: pandas object

    Return:
    ----------
    onsets: pandas object
    """
    # add column headers
    onsets.columns = ["TrialNum", "Condition", "TrialNum_perCondi",
                      "ImageID", "Trial_part", "onsetSec", "durationSec"]
    onsets.drop([0, 1, 2, 3, 4, 5], axis=0, inplace=True)
    return onsets


def cleanRetriev(ret):
    """
    Parameters:
    ----------
    ret: pandas object

    Return:
    ----------
    ret: pandas object
    """
    # Change column headers
    ret.rename(columns={'category': 'old_new',
                        'Stim': 'stim_file',
                        'OldNumber': 'stim_id',
                        'Recognition_ACC': 'recognition_accuracy',
                        'Recognition_RESP': 'recognition_response',
                        'Recognition_RT': 'recognition_responsetime',
                        'Spatial_RESP': 'position_response',
                        'Spatial_RT': 'position_responsetime',
                        'Spatial_ACC(à corriger voir output-encodage)': 'position_accuracy'},
               inplace=True)
    # re-order columns
    cols = ['old_new', 'stim_file', 'stim_id', 'recognition_response',
            'recognition_accuracy', 'recognition_responsetime',
            'position_response', 'position_accuracy', 'position_responsetime']
    ret = ret[cols]
    # Transform reaction time columns from ms to s
    ret[['recognition_responsetime']] = ret[['recognition_responsetime']].astype('float64', copy=False)  # string is object in pandas, str in Python
    ret[['position_responsetime']] = ret[['position_responsetime']].astype('float64', copy=False)
    ret['recognition_responsetime'] = ret['recognition_responsetime'].div(1000)
    ret['position_responsetime'] = ret['position_responsetime'].div(1000)
    # Clean up eprime programming mistake: replace position_response and position_responsetime values
    # with NaN if subject perceived image as 'new' (the image was not probed for position).
    # There should be no response or RT value there, values were carried over from previous trial (not reset in eprime)
    # CONFIRMED w Isabel: subject must give a position answer when probed (image considered OLD) before eprime moves to the next trial.
    i = ret[ret['recognition_response'] == 2].index
    ret.loc[i, 'position_responsetime'] = NaN
    ret.loc[i, 'position_response'] = -1
    # clean up eprime mistake (change Old67 condition ('old_new') from New to OLD)
    q = ret[ret['stim_id'] == 'Old67'].index
    ret.loc[q, 'old_new'] = 'OLD'
    # insert new columns
    colNames = ['trial_number', 'stim_category', 'stim_name',
                'recognition_performance', 'position_correct']
    dtype = [-1, 'None', 'None', 'None', -1]
    colIndex = [0, 4, 5, 9, 10]
    for j in range(0, 5):
        ret.insert(loc=colIndex[j], column=colNames[j], value=dtype[j],
                   allow_duplicates=True)
    # Extract info and fill trial_number, stim_category and stim_name columns
    k = ret.index
    ret.loc[k, 'trial_number'] = k+1
    # format: category_imageName.bmp w some space, _ and - in image names
    stimInfo = ret.loc[k, 'stim_file']
    for s in k:
        ret.loc[s, 'stim_category'] = re.findall('(.+?)_', stimInfo[s])[0]
        ret.loc[s, 'stim_name'] = re.findall('_(.+?)[.]', stimInfo[s])[0]
    # Fill recognition_performance column based on actual and perceived novelty
    m = ret[ret['old_new'] == 'OLD'].index.intersection(ret[ret['recognition_accuracy'] == 1].index)
    ret.loc[m, 'recognition_performance'] = 'Hit'
    n = ret[ret['old_new'] == 'OLD'].index.intersection(ret[ret['recognition_accuracy'] == 0].index)
    ret.loc[n, 'recognition_performance'] = 'Miss'
    o = ret[ret['old_new'] == 'New'].index.intersection(ret[ret['recognition_accuracy'] == 1].index)
    ret.loc[o, 'recognition_performance'] = 'CR'
    p = ret[ret['old_new'] == 'New'].index.intersection(ret[ret['recognition_accuracy'] == 0].index)
    ret.loc[p, 'recognition_performance'] = 'FA'
    # return cleaned up input Dataframe
    return ret


def addOnsets(main, enc):
    """
    Parameters:
    ----------
    main:
    enc: pandas objects

    Return:
    ----------
    main: pandas object
    """

    # make main file indexable by trial number:
    main.set_index('trial_number', inplace=True)
    # copy trial onset and offset times from enc into main
    # note: fixation's onset time is the trial task's offset time
    for i in enc.index:
        trialNum = enc.loc[i, 'TrialNum']
        if enc.loc[i, 'Trial_part'] == 'Fixation':
            main.loc[trialNum, 'offset'] = enc.loc[i, 'onsetSec']
        else:
            main.loc[trialNum, 'onset'] = enc.loc[i, 'onsetSec']
    # Calculate trial duration time from onset and offset times
    main['duration'] = main['offset']-main['onset']
    # reset main's searchable index to default
    main.reset_index(level=None, drop=False, inplace=True)
    return main


def addPostScan(main, ret):
    """
    Parameters:
    ----------
    main: panda object
    ret: panda object

    Return:
    ----------
    mainMerged: pandas object
    """
    # split main's rows (trials) into sublist based on Condition
    mainEnc = main[main['trial_type'] == 'Enc'].copy()
    mainCTL = main[main['trial_type'] == 'CTL'].copy()
    # make mainEnc indexable by picture id
    mainEnc.set_index('stim_id', inplace=True)
    # import post-scan data from ret into mainEnc
    for i in ret[ret['old_new'] == 'OLD'].index:
        stimID = ret.loc[i, 'stim_id']
        mainEnc.loc[stimID, 'stim_category'] = ret.loc[i, 'stim_category']
        mainEnc.loc[stimID, 'stim_name'] = ret.loc[i, 'stim_name']
        mainEnc.loc[stimID, 'recognition_accuracy'] = ret.loc[i, 'recognition_accuracy']
        mainEnc.loc[stimID, 'recognition_responsetime'] = ret.loc[i, 'recognition_responsetime']
        mainEnc.loc[stimID, 'position_response'] = ret.loc[i, 'position_response']
        mainEnc.loc[stimID, 'position_responsetime'] = ret.loc[i, 'position_responsetime']
    # calculate post-scan source (position) accuracy;
    #  -1 = control task; 0 = missed trial; 1 = wrong source (image recognized but wrong quadrant remembered);
    # 2 = image recognized with correct source
    mainEnc['position_accuracy'] = 0
    for j in mainEnc[mainEnc['recognition_accuracy'] == 1].index:
        if mainEnc.loc[j, 'position_correct'] == mainEnc.loc[j, 'position_response']:
            mainEnc.loc[j, 'position_accuracy'] = 2
        else:
            mainEnc.loc[j, 'position_accuracy'] = 1
    # import source accuracy info from mainEnc into ret (in-place)
    for i in ret[ret['old_new'] == 'OLD'].index:
        picID = ret.loc[i, 'stim_id']
        ret.loc[i, 'position_correct'] = mainEnc.loc[picID, 'position_correct']
        ret.loc[i, 'position_accuracy'] = mainEnc.loc[picID,
                                                      'position_accuracy']
    # reset mainEnc searchable index to default
    # and re-order columns to match order in mainCTL
    mainEnc.reset_index(level=None, drop=False, inplace=True)
    cols = ['trial_number', 'onset', 'duration', 'offset', 'trial_type',
            'response', 'response_time', 'stim_id', 'stim_file',
            'stim_category', 'stim_name', 'recognition_accuracy',
            'recognition_responsetime', 'position_correct',
            'position_response', 'position_accuracy', 'position_responsetime']
    mainEnc = mainEnc[cols]
    # Re-merge mainEnc and mainCTL and re-order by trial number
    mainMerged = mainEnc.append(mainCTL, ignore_index=True)
    mainMerged.sort_values('trial_number', axis=0, ascending=True,
                           inplace=True)
    return mainMerged


def extract_taskFile(bID, sID, file_list, output):
    """
    Parameters:
    ----------
    bID: string (subject PSCID, id used during data collection)
    sID: string (subject DCCID, id used in Loris)
    file_list: list (three input files)
    output: string (output Folder)

    Return:
    ----------
    None
    """
    # import data from three text files into pandas DataFrames
    encMain = pd.read_csv(file_list[0], sep='\t')
    manualEdits = ['3303819', '5477234', '6417837', '7674650']
    if bID in manualEdits:
        encOnsets = pd.read_csv(file_list[1], sep='\t', header=None)
    else:
        encOnsets = pd.read_fwf(file_list[1], infer_nrows=210,
                                delim_whitespace=True,
                                header=None)
    retriev = pd.read_csv(file_list[2], sep='\t', encoding='ISO-8859-1')
    # clean up each file
    encMain = cleanMain(encMain)
    encOnsets = cleanOnsets(encOnsets)
    retriev = cleanRetriev(retriev)
    # import onset times from encOnset into encMain
    encMain = addOnsets(encMain, encOnsets)
    # import post-scan performance data from retriev into encMain
    encMain = addPostScan(encMain, retriev)
    # export encMain and retriev into tsv files (output directorty)
    encMain.to_csv(output+'/sub-'+sID+'_ses-4_task-memory_events.tsv',
                   sep='\t', header=True, index=False)
    retriev.to_csv(output+'/PostScanBehav_pscid'+bID+'_dccid'+sID+'.tsv',
                   sep='\t', header=True, index=False)


def main():
    args = get_arguments()
    logging.basicConfig(level=args.log_level)
    oFolder = args.odir[0]
    iFolder = args.idir[0]

    # Create oFolder if not exists
    if not os.path.exists(oFolder):
        os.mkdir(oFolder)

    all_ids = get_all_ids(iFolder)
    # Create tmp folder to temporaly store unziped files
    tmpFolder = join(oFolder, 'tmp')
    if not os.path.exists(tmpFolder):
        os.mkdir(tmpFolder)

    # Create taskFiles folder where all output files will be saved
    fileFolder = join(oFolder, 'taskfiles')
    if not os.path.exists(fileFolder):
        os.mkdir(fileFolder)

    # loop over zip files
    for (idBEH, idMRI) in all_ids:
        s_files = set_subject_data(idBEH, iFolder, tmpFolder)
        if(len(s_files) == 3):
            extract_taskFile(idBEH, idMRI, s_files, fileFolder)
            shutil.rmtree(tmpFolder, ignore_errors=True)
        else:
            logging.info('missing files for subject ({},{})'.format(idBEH,
                                                                    idMRI))

            #


if __name__ == '__main__':
    sys.exit(main())


In [9]:
def sniffzipped(zfolderpath="path/to/folder"):
    '''
    Obtain info about files in zipped archive
     - Without requiring extraction. 
    Parameters
    ----------
        zfolder: Path or path-like object as in os.path module
        
    Returns
    -------
        infos: Dict mapped as {filenameA: infoA, 
                                  [...],
                              filenameZ: infosZ}
    Source: https://chardet.readthedocs.io/en/latest/usage.html                           
    '''
    results = []
    infos = []
    detector = udet()
    with open(zipfile.ZipFile(zfolderpath)) as archv:
        detector.reset
        for line in archv.readlines():
            detector.feed(line)
            if detector.done: break
        infos.append(dict((filename,
                           [archv.getinfo(join(zfolderpath, filename))
                            for file in archv.namelist()])))
        results.append(dict(detector.result))
        detector.close()
        archv.close()
    return tuple(zip(sorted(results), sorted(infos)))
